In [6]:
from statsbombpy import sb
import re
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score,cross_val_predict, KFold
from sklearn.svm import SVR
from xgboost import XGBRegressor, XGBClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, classification_report

In [7]:
#Perform Correlation of all features after all necessary data manipulation

# # Load your dataset
data = pd.read_csv('Statsbomb Data.csv')

# Function to handle unhashable types for one-hot encoding
def make_hashable(x):
    if isinstance(x, list):
        return str(x)  # Convert lists to string to make them hashable
    return x

# Apply this function to all columns that might contain unhashable types
for col in data.columns:
    data[col] = data[col].apply(make_hashable)

# Define functions to calculate additional metrics if relevant
def calculate_distance_to_goal(shot_location):
    if isinstance(shot_location, str):
        shot_location = eval(shot_location)  # Convert string back to list
    goal_x, goal_y = 120, 40  # Coordinates of the center of the goal line
    return np.sqrt((goal_x - shot_location[0])**2 + (goal_y - shot_location[1])**2)

def calculate_angle_to_goal(shot_location):
    if isinstance(shot_location, str):
        shot_location = eval(shot_location)  # Convert string back to list
    goal_y1, goal_y2 = 36, 44  # y-coordinates of the goalposts
    angle1 = np.arctan2(goal_y1 - shot_location[1], 120 - shot_location[0])
    angle2 = np.arctan2(goal_y2 - shot_location[1], 120 - shot_location[0])
    return abs(angle1 - angle2)

# Calculate metrics and add to data
data['distance_to_goal'] = data['location'].apply(calculate_distance_to_goal)
data['angle_to_goal'] = data['location'].apply(calculate_angle_to_goal)

# Convert all categorical variables using one-hot encoding
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# Calculate correlations with 'shot_statsbomb_xg'
correlation_matrix = data.corr()
statsbomb_xg_correlations = correlation_matrix['shot_statsbomb_xg'].sort_values(ascending=False)

# Print the correlations
print(statsbomb_xg_correlations)

# Save to CSV if needed
statsbomb_xg_correlations.to_csv('complete_correlation_with_statsbomb_xg.csv')


KeyError: 'shot_statsbomb_xg'

In [12]:
# Check Correlation of all features expected goals
# Load the dataset
file_path = 'model_data.csv'  # Using the uploaded file
data = pd.read_csv(file_path)

# Encode 'shot_outcome' where 'Goal' = 1 and everything else = 0.
data['shot_outcome_encoded'] = data['shot_outcome'].apply(lambda x: 1 if x == 'Goal' else 0)

# Convert True/False values to 1/0
data = data.replace({True: 1, False: 0})

# Drop columns that are not needed in the correlation analysis
columns_to_drop = ['statsbomb_xg', 'shot_outcome', 'player', 'possession_team', 'location', 'goalkeeper_location']
data = data.drop(columns=columns_to_drop)

# Apply one-hot encoding to categorical variables
data_encoded = pd.get_dummies(data, drop_first=True)

# Calculate the correlation matrix
correlation_matrix = data_encoded.corr()

# Extract the correlation of all features with 'shot_outcome_encoded'
correlation_with_shot_outcome = correlation_matrix['shot_outcome_encoded'].sort_values(ascending=False)

# Get the top 15 features with highest absolute correlation (positive or negative)
top_25_correlated_features = correlation_with_shot_outcome.abs().sort_values(ascending=False).head(25)

# Display the top 25 correlated features
top_25_correlated_features


shot_outcome_encoded           1.000000
play_pattern_Penalty           0.384713
angle_to_goal                  0.277889
distance_to_goal               0.222281
goalkeeper_distance            0.206335
shot_speed                     0.113100
defensive_pressure             0.106792
play_pattern_From Free Kick    0.090469
shot_body_part_Other           0.083420
minute                         0.074779
shot_one_on_one                0.063426
play_pattern_From Throw In     0.058788
shot_first_time                0.033440
play_pattern_From Kick Off     0.032149
play_pattern_From Counter      0.031794
position_Goalkeeper            0.030311
play_pattern_Regular Play      0.025677
shot_body_part_Head            0.018608
play_pattern_From Keeper       0.016566
position_Midfielder            0.010968
play_pattern_From Goal Kick    0.003354
position_Forward               0.000659
Name: shot_outcome_encoded, dtype: float64

In [ ]:
#CODE TO EXTRACT relevant DATA based on correlation

# # get a list of all competitions
competitions = sb.competitions()
competitions = competitions[competitions['competition_gender'] == 'female']

#using the season id and competition id from competitions, get all match ids from competition
matches = sb.matches(competition_id=72, season_id=107)
match_ids = matches['match_id']
match_ids

# filter shots data to only include columns of interest
cols = ['id', 'player', 'team', 'minute', 'location', 'shot_type', 'shot_outcome', 'under_pressure', 'shot_body_part', 'shot_end_location',
        'shot_first_time', 'shot_freeze_frame', 'shot_key_pass_id', 'shot_one_on_one', 'shot_outcome', 'shot_technique', 'shot_first_time', 'shot_deflected', 'pass_assisted_shot_id', 
        'pass_shot_assist', 'pass_goal_assist', 'pass_type','pass_end_location', 'pass_assisted_shot_id', 'goalkeeper_end_location', 'goalkeeper_position',
        'goalkeeper_outcome', 'goalkeeper_technique', 'period', 'play_pattern', 'shot_statsbomb_xg']

#for each match, get all shots data
shots_data = pd.DataFrame()
for i in range(0, len(match_ids)):
    #try:
    print(match_ids[i])
    match_events = sb.events(match_id=match_ids[i])
    shots = match_events[match_events['type'] == 'Shot']
    #shots = shots[cols]
    shots_data = pd.concat([shots_data, shots])
shots_df = pd.DataFrame(shots_data)
shots_df.to_csv("Statsbomb Data.csv")


In [ ]:
#VIF analysis for understanding multicollinearity

# Load the processed dataset
file_path = 'model_data.csv'
data = pd.read_csv(file_path)

# Create the target variable based on shot_outcome
data['shot_outcome'] = data['shot_outcome'].apply(lambda x: 1 if x == 'Goal' else 0)

# Drop the columns that are not needed for the model
data = data.drop(columns=['distance_to_goal', 'angle_to_goal','location', 'goalkeeper_location', 'goalkeeper_distance','under_pressure', 'statsbomb_xg', 'possession_team', 'player'])

# Handle categorical variables including 'position'
data = pd.get_dummies(data, drop_first=True)

# Define the features and target variable
X = data.drop(columns=['shot_outcome'])
y = data['shot_outcome']

# Ensure all values in X are numerical and finite
X = X.apply(pd.to_numeric, errors='coerce')
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

# Re-check and convert data types if necessary
X = X.astype(float)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display the VIF data
print(vif_data)

# Check for high VIF values
high_vif_features = vif_data[vif_data['VIF'] > 5]
if not high_vif_features.empty:
    print("Features with high VIF:")
    print(high_vif_features)
else:
    print("No features with high VIF detected.")

#REMOVED play_pattern (shot_type) and shot_technique (shot_body_part) cos of high multicollinearity


In [3]:


# Load the processed dataset
file_path = 'Statsbomb Data.csv'
data = pd.read_csv(file_path)

# Define the goal post coordinates considering the goal width
goal_center_x = 120
goal_center_y = 40.3
goal_width = 7.32
goalkeeper_center_location = [goal_center_x, goal_center_y]  # Goalkeeper's position for penalties

def calculate_distance_to_goal(location):
    x, y = location
    return np.sqrt((goal_center_x - x) ** 2 + (goal_center_y - y) ** 2)

def calculate_angle_to_goal_posts(location):
    x, y = location
    left_post_angle = np.arctan2(goal_center_y - (goal_width / 2) - y, goal_center_x - x)
    right_post_angle = np.arctan2(goal_center_y + (goal_width / 2) - y, goal_center_x - x)
    return np.degrees(right_post_angle - left_post_angle)

def normalize_shot_body_part(shot_body_part):
    if shot_body_part in ['Left Foot', 'Right Foot']:
        return 'Foot'
    return shot_body_part

def classify_position(position):
    position_dict = {
        'Forward': ['Center Forward', 'Left Wing', 'Right Wing', 'Left Center Forward', 'Right Center Forward'],
        'Midfielder': ['Center Attacking Midfield', 'Left Midfield', 'Right Midfield', 'Left Center Midfield', 'Right Center Midfield', 'Center Defensive Midfield', 'Left Defensive Midfield', 'Right Defensive Midfield', 'Left Attacking Midfield', 'Right Attacking Midfield'],
        'Defender': ['Left Back', 'Right Back', 'Left Center Back', 'Right Center Back', 'Center Back', 'Left Wing Back', 'Right Wing Back'],
        'Goalkeeper': ['Goalkeeper']
    }

def calculate_distance(location1, location2):
    if not location1 or not location2:
        return np.nan
    return np.sqrt((location1[0] - location2[0])**2 + (location1[1] - location2[1])**2)

def calculate_defensive_pressure(shot_location, opponent_locations):
    return sum(1 for loc in opponent_locations if calculate_distance(shot_location, loc) <= 5)

# Extract and handle goalkeeper location and opponent locations
def extract_goalkeeper_and_opponents(freeze_frame):
    opponents = []
    goalkeeper = None
    if isinstance(freeze_frame, str):
        pattern = r"\{'location': \[([\d.]+), ([\d.]+)\], 'player': .+?, 'position': \{'id': \d+, 'name': '([^']+)'(?:.*?)\}, 'teammate': (False|True)\}"
        matches = re.finditer(pattern, freeze_frame)
        for match in matches:
            location = eval(f"[{match.group(1)}, {match.group(2)}]")
            position_name = match.group(3)
            teammate = match.group(4) == 'True'
            if not teammate:
                if position_name == 'Goalkeeper':
                    goalkeeper = location
                else:
                    opponents.append(location)
    return goalkeeper if goalkeeper else goalkeeper_center_location, opponents

data['goalkeeper_location'], data['opponent_locations'] = zip(*data['shot_freeze_frame'].apply(extract_goalkeeper_and_opponents))

# Calculate metrics
data['goalkeeper_distance'] = data.apply(
    lambda x: calculate_distance(eval(x['location']), x['goalkeeper_location']) if x['location'] and x['goalkeeper_location'] else np.nan,
    axis=1
).round(5)

# Convert string representation of location to tuple if it's a string
data['location'] = data['location'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Calculate additional metrics
data['defensive_pressure'] = data.apply(lambda x: calculate_defensive_pressure(x['location'], x['opponent_locations']), axis=1)
data['distance_to_goal'] = data['location'].apply(calculate_distance_to_goal).round(5)
data['angle_to_goal'] = data['location'].apply(calculate_angle_to_goal_posts).round(5)
data['shot_speed'] = (data['distance_to_goal'] / data['duration']).round(5)
data['shot_body_part'] = data['shot_body_part'].apply(normalize_shot_body_part)
data['position'] = data['position'].apply(classify_position)

# Rename the 'shot_statsbomb_xg' column to 'statsbomb_xg'
data.rename(columns={'shot_statsbomb_xg': 'statsbomb_xg'}, inplace=True)

# Save the processed data
processed_data = data[['possession_team', 'player', 'position', 'location', 'distance_to_goal', 'angle_to_goal', 'shot_speed', 'play_pattern', 'shot_type', 'minute', 'period', 'under_pressure', 'shot_body_part', 'shot_one_on_one', 'shot_first_time', 'shot_technique', 'shot_outcome', 'goalkeeper_location', 'goalkeeper_distance', 'defensive_pressure', 'statsbomb_xg']]

#create the dataset which is used in the machine learning models
processed_data.to_csv('model_data.csv', index=False)

# # Display the first few rows
# processed_data.head()
